In [1]:
import sys
sys.path.append("../")
from src.utils.data_util import biocaser2text
from src.eval_method import CrossValidation
from src.experiment import Experiment
from src.metrics import Accuracy,Precision,Recall,F1Score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from src.preprocess import BasicTextPreprocess,BasicTextFeatureExtraction

In [2]:
## this should load a dataframe, which col1 being the documents and col2 being the labels.
from utils.data_util import get_raw_promed_df
data_df = get_raw_promed_df()

found 3009 files
found files: 3009
found 1530 files
found files: 1530


In [3]:
data_df.labels[1]

1

In [4]:
bp = BasicTextPreprocess()

bfe = BasicTextFeatureExtraction(["tfidf","bow","bigram",["tfidf","bigram"],["bow","bigram"],["tfidf","trigram"],["bow","trigram"],["tfidf","bigram","trigram"]])

cv = CrossValidation(n_splits=10, n_repeats=10, random_state=12345)

acc = Accuracy()
pre = Precision()
rec = Recall()
f1 = F1Score()

svm = LinearSVC(random_state=0, tol=1e-5,max_iter=5000)
svm.name = "svm"
dtree = DecisionTreeClassifier(random_state=0,criterion='entropy')
dtree.name="dtree"
bnb = BernoulliNB()
bnb.name = "BernoulliNB"
gnb = GaussianNB()
gnb.name = "GaussianNB"
mnb = MultinomialNB()
mnb.name = "MultinomialNB"
rf = RandomForestClassifier(max_depth=2, random_state=0)
rf.name = "RandomForest"

# chi2 feature selection
ch2 = SelectKBest(chi2, k=9000)
svm_ch2 = Pipeline([('sel', ch2), ('cls', svm)])
svm_ch2.name = "svm+chi2"

# chi2 feature selection
dtree_ch2 = Pipeline([('sel', ch2), ('cls', dtree)])
dtree_ch2.name = "dtree+chi2"

# chi2 feature selection
bnb_ch2 = Pipeline([('sel', ch2), ('cls', bnb)])
bnb_ch2.name = "BernoulliNB+chi2"

# chi2 feature selection
gnb_ch2 = Pipeline([('sel', ch2), ('cls', gnb)])
gnb_ch2.name = "GaussianNB+chi2"

# chi2 feature selection
mnb_ch2 = Pipeline([('sel', ch2), ('cls', mnb)])
mnb_ch2.name = "MultinomialNB+chi2"

# chi2 feature selection
mnb_rf = Pipeline([('sel', ch2), ('cls', rf)])
mnb_rf.name = "RandomForest+chi2"

In [5]:
from src.models.linear import RandomWord2vec
randomw2c = RandomWord2vec(data_df)
randomw2c.name = "RandomWord2vec"

In [6]:
exp = Experiment(
    data_df = data_df,
#     preprocessor = bp,
    preprocessor = None,
#     extractor = bfe,
    extractor = None,
    eval_method=cv,
#     models=[svm,dtree,bnb,gnb,mnb,svm_ch2,dtree_ch2,bnb_ch2,gnb_ch2,mnb_ch2],
    models = [randomw2c],
    metrics=[acc,pre,rec,f1],
    result_file = "../promed_result.csv",
    verbose = True
).run()

0it [00:00, ?it/s]

Found feature_columns: ['docs']
Experiment with RandomWord2vec model on docs feature.


/home/zm324/anaconda3/envs/doc_cls/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


Epoch: 1  | time in 0 minutes, 0 seconds
	Loss: 0.0045(train)	|	Acc: 70.2%(train)
Epoch: 2  | time in 0 minutes, 0 seconds
	Loss: 0.0032(train)	|	Acc: 86.0%(train)
Epoch: 3  | time in 0 minutes, 0 seconds
	Loss: 0.0023(train)	|	Acc: 93.5%(train)
Epoch: 4  | time in 0 minutes, 0 seconds
	Loss: 0.0019(train)	|	Acc: 94.8%(train)
Epoch: 5  | time in 0 minutes, 0 seconds
	Loss: 0.0016(train)	|	Acc: 95.0%(train)
Epoch: 6  | time in 0 minutes, 0 seconds
	Loss: 0.0014(train)	|	Acc: 95.3%(train)
Epoch: 7  | time in 0 minutes, 0 seconds
	Loss: 0.0013(train)	|	Acc: 95.5%(train)
Epoch: 8  | time in 0 minutes, 0 seconds
	Loss: 0.0012(train)	|	Acc: 95.7%(train)
Epoch: 9  | time in 0 minutes, 0 seconds
	Loss: 0.0012(train)	|	Acc: 95.8%(train)
Epoch: 10  | time in 0 minutes, 0 seconds
	Loss: 0.0011(train)	|	Acc: 96.0%(train)
Epoch: 11  | time in 0 minutes, 0 seconds
	Loss: 0.0011(train)	|	Acc: 95.9%(train)
Epoch: 12  | time in 0 minutes, 0 seconds
	Loss: 0.0011(train)	|	Acc: 96.0%(train)
Epoch: 13  | 

0it [00:07, ?it/s]

Epoch: 50  | time in 0 minutes, 0 seconds
	Loss: 0.0008(train)	|	Acc: 96.7%(train)


KeyError: 1

In [ ]:
data_df.iloc[0].docs

In [ ]:
data_df["labels"].value_counts()